In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

print("Pytorch: {}".format(torch.__version__))

Pytorch: 0.1.12_2


# Load datasets

In [2]:
mnist_data_path = './data/pytorch_data'
transform = torchvision.transforms.ToTensor() # transform PIL image to Tensor
mnist_train = torchvision.datasets.MNIST(mnist_data_path, train=True, transform=transform,
                                         target_transform=None, download=True)
mnist_test  = torchvision.datasets.MNIST(mnist_data_path, train=False, transform=transform,
                                         target_transform=None,download=True)
print("train_len {}, test_len {}".format(len(mnist_train), len(mnist_test)))

train_len 60000, test_len 10000


In [3]:
# See one example
sample = mnist_train[0]
print("Sample shape: {}".format(sample[0][0].size()))
print("Sample target: {}".format(sample[1]))
torchvision.transforms.ToPILImage()(sample[0])

Sample shape: torch.Size([28, 28])
Sample target: 5


In [4]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=32)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=32)
# loader speedup could be achieved by following parameters
# test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1024,
#                                           pin_memory=True, num_workers=4, shuffle=False)

# Model definition with sequential nn.Module (easy way)

In [5]:
class NnModuleSequential(nn.Module):
    def __init__(self, img_size):
        super(NnModuleSequential,self).__init__()
        
        self.conv1 = nn.Sequential(
                nn.Conv2d(1, 32, kernel_size=5),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(32, 64, kernel_size=5),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),
        )
        
        self.fc1 = nn.Linear(int((((img_size-4)/2-4)/2)**2*64), 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = out.view(x.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        return out

### Testing that the initial model works

In [6]:
model = NnModuleSequential(img_size=28).cuda()

In [7]:
x = Variable(mnist_train[0][0].unsqueeze(0))
x = x.cuda()
model(x)

Variable containing:
1.00000e-02 *
 -1.9160 -4.7896 -3.2303  1.0539 -3.2672  0.2201  3.2853  2.7627  2.1575 -7.0689
[torch.cuda.FloatTensor of size 1x10 (GPU 0)]

### Accuracy testing

In [8]:
def test_model_accuracy(model, data_loader, use_cuda=False):
    model.eval() # Set model in evaluation mode so dropout etc won't affect
    test_loss, correct = 0, 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if use_cuda:
            data, target = data.cuda(), target.cuda()
            
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(data_loader.dataset)
    print('Test set: Average loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)'.format(
                test_loss, correct, len(data_loader.dataset),
                100. * correct / len(data_loader.dataset)))

In [9]:
# We now have model with random accuracy
test_model_accuracy(model, test_loader, use_cuda=True)

Test set: Average loss: 2.308021, Accuracy: 1029/10000 (10.29%)


In [10]:
%%timeit -r 3
test_model_accuracy(model, test_loader, use_cuda=True)

Test set: Average loss: 2.308021, Accuracy: 1029/10000 (10.29%)
Test set: Average loss: 2.308021, Accuracy: 1029/10000 (10.29%)
Test set: Average loss: 2.308021, Accuracy: 1029/10000 (10.29%)
Test set: Average loss: 2.308021, Accuracy: 1029/10000 (10.29%)
667 ms ± 1.78 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Training the model

In [11]:
def train_epoch(model, data_loader, optimizer, log_interval_percentage=.35, use_cuda=False):
    model.train()
    train_loss = []
    log_interval = int((len(data_loader.dataset)/data_loader.batch_size) * log_interval_percentage)
    
    for batch_idx, (data, target) in enumerate(data_loader):
        data = Variable(data)
        target = Variable(target)
        if use_cuda:
            data, target = data.cuda(), target.cuda()

        predict = model(data)
        loss = F.cross_entropy(predict, target)
        train_loss.append(loss.data[0])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx % log_interval == 0) and (batch_idx > 0):
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(data_loader.dataset),
                        100. * batch_idx / len(data_loader), np.average(train_loss)))
    print(("Epoch: {}, Loss: {:.6f}").format(epoch , np.average(train_loss)))

In [12]:
total_epochs = 0

In [13]:
%%time
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(3):
    total_epochs += 1
    train_epoch(model, train_loader, optimizer, use_cuda=True)
    test_model_accuracy(model, test_loader, use_cuda=True)
print()

Epoch: 0 [20992/60000 (35%)]	Loss: 0.202691
Epoch: 0 [41984/60000 (70%)]	Loss: 0.139142
Epoch: 0, Loss: 0.116007
Test set: Average loss: 0.064403, Accuracy: 9795/10000 (97.95%)
Epoch: 1 [20992/60000 (35%)]	Loss: 0.052545
Epoch: 1 [41984/60000 (70%)]	Loss: 0.049709
Epoch: 1, Loss: 0.046479
Test set: Average loss: 0.074695, Accuracy: 9770/10000 (97.70%)
Epoch: 2 [20992/60000 (35%)]	Loss: 0.036201
Epoch: 2 [41984/60000 (70%)]	Loss: 0.036093
Epoch: 2, Loss: 0.035140
Test set: Average loss: 0.040398, Accuracy: 9875/10000 (98.75%)

CPU times: user 19.1 s, sys: 76 ms, total: 19.2 s
Wall time: 19.2 s


# Save & Load & Predict

### Saving

In [14]:
model_dir = './data/pytorch_model'
import os
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [15]:
saved_model_path = model_dir + '/full_model_' + str(total_epochs)
saved_model_path

'./data/pytorch_model/full_model_3'

In [16]:
torch.save(model.state_dict(), saved_model_path)

### Loading

In [17]:
loaded_model = NnModuleSequential(img_size=28)
loaded_model.load_state_dict(torch.load(saved_model_path))
loaded_model = loaded_model.cuda()

### Predicting

In [18]:
sample_idx = int(random.random()*100)
sample = mnist_test[sample_idx][0]
x = Variable(sample.unsqueeze(0))
x = x.cuda()

result = loaded_model(x)

result_class = result.max(1)[1].data[0][0]
result_correct = result_class == mnist_test[sample_idx][1]
result_softmax = F.softmax(result[0])[result_class].data[0]
print("{} (softmax: {}, correct: {})".format(result_class, result_softmax, result_correct))
torchvision.transforms.ToPILImage()(sample)

4 (softmax: 0.9991902709007263, correct: True)


# Debugging

In [19]:
print("Easier way to see what the output size of some sequential layer is, make one forward")
img_size = 28
test_model = NnModuleSequential(img_size=img_size)
output_shape = test_model.conv1(Variable(torch.ones(1, 1, img_size, img_size))).size()
print("Output shape: {}".format(output_shape))
print("Total params: {}".format(np.prod(output_shape)))

Easier way to see what the output size of some sequential layer is, make one forward
Output shape: torch.Size([1, 64, 4, 4])
Total params: 1024
